### Defines Devices calcPotInst(), calcSNR() e decodifica()

In [ ]:
import numpy as np
from random import uniform

class Device:
    def __init__(self,r_min, r_max, power,k, power_ref, delta,control):
        
        self.r_max = r_max
        self.r_min = r_min
        self.power = power
        self.power_max = power
        self.PrControl = 0
        self.randomizePlace()
        if control:
          self.setPowerTransmitControl(fc=915e6,n=3,pr_dBm = power_ref)
        self.Q_Table = 2*np.random.random_sample((3,k)) - 1


        self.estado_delta = 0 
    
    def setX(self, x):
        self.x = x
     
    def setY(self, y):
        self.y = y
     
    def getX(self):
        return self.x
         
    def getY(self):
        return self.y
    
    def getR(self):
        return self.r
    
    def randomizePlace(self):
        arg = 2*np.pi*uniform(0,1)
        self.r = self.r_max*np.sqrt(uniform(self.r_min,1))
        self.x = self.r*np.cos(arg)
        self.y = self.r*np.sin(arg)
        
    def setPrControl(self,Pr_ctrl):
        self.PrControl = Pr_ctrl
        
        
    def getPr(self,n,fc,control=False):
        
        c = 3e8
        d0 = 1
        pt = self.power  #power transmitted
        d=self.r*1e3

        pl_d0 = -20*np.log10(c/(fc*4*np.pi*d0))

        pr_d0 = pt-pl_d0

        pr = pr_d0 - 10*n*np.log10(d/d0)

        pr_dBm = pr+30

        return pr_dBm

    def setEstadoDelta(self, estado):
      self.estado_delta = estado

    def setPowerTransmitControl(self, n,fc,pr_dBm):

      c = 3e8
      d0 = 1
      d=self.r*1e3
      pl_d0 = -20*np.log10(c/(fc*4*np.pi*d0))
      pt = pr_dBm - 30 + 10*n*np.log10(d/d0) + pl_d0

      self.power = pt

    def setPt(self, pt):

      self.power = pt


def calcPotInst(PotMedia):
    
    PotInst = np.random.rayleigh(PotMedia, 1)
     
    return PotInst
def calcSNR(dev_index, devices,r,noma,fading,control):
    
    ctrl = control
    SINR = np.zeros(len(dev_index))
    PowerRec = np.zeros(len(dev_index))
    
    for i,d in enumerate(dev_index):
        if(fading):
            PowerRec[i] = calcPotInst(10**((devices[d].getPr(3,915e6,ctrl)-30)/10)) # potencia em watts
        else:
            PowerRec[i] = 10**((devices[d].getPr(3,915e6,ctrl)-30)/10)
            
   #print(PowerRec)

    refPowerRec = PowerRec
    falha = False
    for i,d in enumerate(dev_index):
        if noma:
            
            SINR[np.where(refPowerRec == np.amax(PowerRec))] = np.amax(PowerRec)/(np.sum(PowerRec[PowerRec != np.amax(PowerRec)]) + 1.58e-15)
            if (SINR[np.where(refPowerRec == np.amax(PowerRec))] < (2**r -1) or falha == True):
                falha = True
                SINR[np.where(refPowerRec == np.amax(PowerRec))] = -1000
                
                
            PowerRec = PowerRec[PowerRec != np.amax(PowerRec)]
            
        else:
            if (i==0):
                SINR[np.where(refPowerRec == np.amax(PowerRec))] = np.amax(PowerRec)/(np.sum(PowerRec[PowerRec != np.amax(PowerRec)]) + 1.58e-15)
            else:
                SINR[np.where(refPowerRec == np.amax(PowerRec))] = -1000
            PowerRec = PowerRec[PowerRec != np.amax(PowerRec)]
    return SINR


def decodifica(slots_escolhidos, devices, r,noma,fading,control):
    dev_success = np.zeros(len(devices))
    k=0
    slots_ref = slots_escolhidos
    
    while(len(slots_escolhidos) > 0):
        
        k +=1

        
        dev_index = np.where(slots_ref == slots_escolhidos[0])[0]
        
        SNR = calcSNR(dev_index,devices,r,noma,fading,control)
        for j,snr in enumerate(SNR):
            if (snr > (2**r - 1)):
                dev_success[dev_index[j]] = 1
   
        slots_escolhidos = slots_escolhidos[slots_escolhidos != slots_escolhidos[0]]
        
    return dev_success

### Rate and Reach

In [ ]:
def define_alcance(r):
    
  c = 3e8
  fc = 915e6
  n=3
  d0 = 1
  pt = -20#-20db

  pn = 1.58e-15

  pl_d0 = -20*np.log10(c/(fc*4*np.pi*d0))

  pr_d0 = pt-pl_d0

  pr_db = 10*np.log10(pn*(2**r-1))
  print(pr_db)
  alcance = ((10**((pr_db - pr_d0)/(-10*n)))/d0)/1e3

  return alcance

In [ ]:
def define_alcance_controle(pr_db):
  c = 3e8
  fc = 915e6
  n=3
  d0 = 1
  pt = -20#-20db
  pn = 1.58e-15

  pl_d0 = -20*np.log10(c/(fc*4*np.pi*d0))

  pr_d0 = pt-pl_d0

  alcance = ((10**((pr_db - pr_d0)/(-10*n)))/d0)/1e3

  return alcance

### Aloha

In [ ]:
%%time
import numpy as np
import matplotlib.pyplot as plt
def alohaArtigo(devices, k,l):

      tpt = np.zeros(len(devices))
      colisao_total = np.zeros(len(devices))
      for i,dev in enumerate(devices):
          for count in range(l):
          
              frame = np.random.randint(1,k+1,dev)
              uniqueValues = np.unique(frame)
              uniqueValues, occurCount = np.unique(frame, return_counts=True)
              sucesso=np.count_nonzero(occurCount == 1)
              colisao = 0
              for col in range(len(occurCount)):
                  if occurCount[col] > 1:
                      colisao += occurCount[col]
              tpt[i] += sucesso/k
              colisao_total[i] += colisao/k

      tpt = tpt/l
      colisao_total = colisao_total/l

      tpt3 =tpt

      colisao1 = colisao_total

      return tpt3

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 14.1 µs


### Aloha with Power decoding main function


In [ ]:
%%time
import numpy as np
import matplotlib.pyplot as plt

def aloha_power_decoding(devices,alcance, taxa, k, l, noma, fading, control):

      tpt = np.zeros(len(devices))
      colisao_total = np.zeros(len(devices))

      for i,dev in enumerate(devices):
          
          for count in range(l):
          
              frame = np.random.randint(1,k+1,dev)
              dispositivos = []
              
              for j in range(dev):
                  
                  disp = Device(r_min = 0.01,r_max = alcance, power = -20, k=k, power_ref= -130, delta=7,control=control )
                  disp.setPt(-20)
                  dispositivos.append(disp)
                  
              sucesso = np.sum(decodifica(frame,dispositivos,r=taxa,noma=noma,fading=fading,control=control))
              tpt[i] += sucesso/k

      tpt = tpt/l
      tpt2_fad = tpt
      return tpt2_fad

CPU times: user 11 µs, sys: 2 µs, total: 13 µs
Wall time: 15.5 µs


### Q_Learning Main Function



In [ ]:
%%time
import numpy as np
import matplotlib.pyplot as plt

def qlearning_simulation(devices,alcance,taxa, k, l, alpha, gamma, noma, fading, control,cong_level,pr_control_dBm,delta_db):

      tpt = np.zeros(len(devices))
      tpt_novo = np.zeros(len(devices))
      colisao_total = np.zeros(len(devices))
      pw_mean = np.zeros(len(devices))

      for i,dev in enumerate(devices):
          
          Q_Values = np.zeros((dev,k))
          # Q_Values = 2*np.random.random_sample((dev,k)) - 1
          
          dispositivos = []
          for j in range(dev):
                  disp = Device(r_min = 0.01,r_max = alcance, power = -20, k=k, power_ref= pr_control_dBm, delta=delta_db,control=control )
                  # if(control == False):
                  #   disp.setPt(-20)
                  # else:
                  #   disp.setPowerTransmitControl(fc=915e6,n=3,pr_dBm = pr_control_dBm-delta_db)

                  dispositivos.append(disp)
          
          for count in range(l):        
              
              frame = np.zeros(dev).astype(int)
              
              for g in range(dev):
                  
                  result = np.where(Q_Values[g,:] == np.max(Q_Values[g,:]))
                  select = np.random.choice(result[0])
                  frame[g] = select
              
              dev_sucesso = decodifica(frame,dispositivos,r=taxa,noma=noma,fading=fading,control=control)

              for q in range(dev):
                  
                  if dev_sucesso[q] == 1:
                      
                      Q_Values[q,frame[q]] = Q_Values[q,frame[q]] + alpha*(1-Q_Values[q,frame[q]]+ gamma*np.max(Q_Values[q,:]))
                      
                  else:
                    ### recompensa artigo original ###
                      if(cong_level):
                          CL = (frame == frame[q]).sum()
                          Q_Values[q,frame[q]] = Q_Values[q,frame[q]] + alpha*(-CL/dev-Q_Values[q,frame[q]]+ gamma*np.max(Q_Values[q,:]))
                    ### recompensa normal r = - 1 ###
                      else:
                          Q_Values[q,frame[q]] = Q_Values[q,frame[q]] + alpha*(-1-Q_Values[q,frame[q]]+ gamma*np.max(Q_Values[q,:]))

              sucesso = np.sum(dev_sucesso)
              tpt[i] += sucesso/k
              tpt_novo[i] += sucesso/dev

          soma_power = 0
          for d in range(dev):
            soma_power += dispositivos[d].power
            # if(dispositivos[d].power > -20):
            #   print(dispositivos[d].r,dispositivos[d].power)

          # print("Pt média:",soma_power/dev)
          pw_mean[i] = soma_power/dev

      # for d in range(dev):
      #   print(dispositivos[d].getPr(n=3,fc=915e6,control=True))
      tpt = tpt/l
      tpt_novo = tpt_novo/l
      return tpt, pw_mean

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 14.1 µs


### Q_learning power control

In [ ]:
%%time
import numpy as np
import matplotlib.pyplot as plt

def qlearning_simulation_delta(devices,alcance,taxa, k, l, alpha, gamma, noma, fading, control,cong_level,pr_control_dBm,delta_db,rwd_pw):

      tpt = np.zeros(len(devices))
      tpt_novo = np.zeros(len(devices))
      colisao_total = np.zeros(len(devices))
      pw_mean = np.zeros(len(devices))

      for i,dev in enumerate(devices):
                    
          dispositivos = []
          for j in range(dev):
                  disp = Device(r_min = 0.01,r_max = alcance, power = -20, k=k,power_ref=pr_control_dBm,delta=delta_db,control=control)
                  dispositivos.append(disp)
          
          for count in range(l):        
              
              frame = np.zeros(dev).astype(int)
              
              for g in range(dev):
                  
                  result = np.where(dispositivos[g].Q_Table == np.max(dispositivos[g].Q_Table))
                  options = np.arange(len(result[0]))
                  select = np.random.choice(options)
                  slot = result[1][select]
                  estado_pr = result[0][select]
                  frame[g] = slot
                  dispositivos[g].setEstadoDelta(estado_pr)

                  if (estado_pr == 0):
                    dispositivos[g].setPowerTransmitControl(fc=915e6,n=3,pr_dBm = pr_control_dBm-delta_db)
                  elif (estado_pr == 1):
                    dispositivos[g].setPowerTransmitControl(fc=915e6,n=3,pr_dBm = pr_control_dBm)
                  else:
                    dispositivos[g].setPowerTransmitControl(fc=915e6,n=3,pr_dBm = pr_control_dBm+delta_db)
              
              dev_sucesso = decodifica(frame,dispositivos,r=taxa,noma=noma,fading=fading,control=control)

              for q in range(dev):
                  
                  if dev_sucesso[q] == 1:
                      #recompensa pela potencia 10/pt_dbm
                      if(rwd_pw):
                        dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]] = dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]] + alpha*((10/(dispositivos[q].power+30))-dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]]+gamma*np.max(dispositivos[q].Q_Table))
                      else: #recompensa comum = 1
                        dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]] = dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]] + alpha*(1-dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]]+gamma*np.max(dispositivos[q].Q_Table))  
                  else:
                      dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]] = dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]] + alpha*(-1-dispositivos[q].Q_Table[dispositivos[q].estado_delta,frame[q]]+gamma*np.max(dispositivos[q].Q_Table)) 

              sucesso = np.sum(dev_sucesso)
              tpt[i] += sucesso/k
              tpt_novo[i] += sucesso/dev

          # print(dev,":\n")
          soma_power = 0
          for d in range(dev):
            soma_power += dispositivos[d].power
            if(dispositivos[d].power > -20):
              print(dispositivos[d].r,dispositivos[d].power)

          # print("Pt média:",soma_power/dev)
          pw_mean[i] = soma_power/dev              

      # print(pw_mean/len(devices))
      tpt = tpt/l
      tpt_novo = tpt_novo/l
      return tpt, pw_mean

CPU times: user 18 µs, sys: 0 ns, total: 18 µs
Wall time: 21.5 µs
